<a href="https://colab.research.google.com/github/yotamgardosh/LLM-Research-Technion/blob/main/llama_cpp_server_harness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ctransformers
!pip install ctransformers[cuda]
!pip install lm-eval
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install llama-cpp-python
!pip install langchain
!pip install langchain-community
!pip install langchain llama-cpp-python
!pip install xlsxwriter
!pip install openpyxl
!pip install huggingface-hub


In [2]:
from datasets import load_dataset
from datasets import load_dataset
from huggingface_hub import notebook_login
from openpyxl import load_workbook
from openpyxl.formatting.rule import CellIsRule
from openpyxl.styles import PatternFill
import pandas as pd
import os
from langchain import LLMChain, PromptTemplate
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.llms import LlamaCpp
from langchain.schema import SystemMessage, HumanMessage
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.llms import LlamaCpp
from langchain.schema import SystemMessage, HumanMessage
!rm -rf /content/LLM-Research-Technion
!git clone https://github.com/yotamgardosh/LLM-Research-Technion.git
os.chdir('LLM-Research-Technion')
from models_class import Model

Cloning into 'LLM-Research-Technion'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 92 (delta 37), reused 34 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 8.95 MiB | 18.19 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [ ]:

# # Use the correct file name from the model repository
# hf_hub_download(repo_id="TheBloke/OpenHermes-2.5-Mistral-7B-GGUF", filename="openhermes-2.5-mistral-7b.Q4_K_M.gguf", local_dir="./models")



openhermes-2.5-mistral-7b.Q4_K_M.gguf:  86%|########6 | 3.76G/4.37G [00:00<?, ?B/s]

'models/openhermes-2.5-mistral-7b.Q4_K_M.gguf'

In [5]:
mistral_model = Model("TheBloke/OpenHermes-2.5-Mistral-7B-GGUF", "Mistral", "openhermes-2.5-mistral-7b.Q4_K_M.gguf")
orca_2_7b = Model("TheBloke/Orca-2-7B-GGUF","Orca2_7b","orca-2-7b.Q4_K_M.gguf")
llama_2_7b = Model("TheBloke/Llama-2-7B-GGUF","Llama2_7b","llama-2-7b.Q4_K_M.gguf")

local_dir = "/content/Models"
# Call create_model_path and pass custom arguments
model_path = mistral_model.create_model_path(
    repo_id=mistral_model.model_name,
    file_name=mistral_model.file_path,
    local_dir=local_dir,
    new_file_name=f"{mistral_model.shorter_name}.gguf"
)


# For Orca-2-7B model
orca_2_7b_path = orca_2_7b.create_model_path(
    repo_id=orca_2_7b.model_name,
    file_name=orca_2_7b.file_path,
    local_dir=local_dir,
    new_file_name=f"{orca_2_7b.shorter_name}.gguf"
)

# For Llama-2-7B model
llama_2_7b_path = llama_2_7b.create_model_path(
    repo_id=llama_2_7b.model_name,
    file_name=llama_2_7b.file_path,
    local_dir=local_dir,
    new_file_name=f"{llama_2_7b.shorter_name}.gguf"
)







------------------------------------------------------------------------
Initialized model 'Mistral' with the following settings:
Context length: 16000
Threads: 32
GPU layers: 0
Max tokens: 30
Temperature: 0.2
Top-p sampling: 0.6

Available Methods:
  - create_model_path(): Download model files to a specified path.
  - create_llm(): Create and configure a language model for use.
  - __str__(): Return a string representation of the model's settings.
------------------------------------------------------------------------



------------------------------------------------------------------------
Initialized model 'Orca2_7b' with the following settings:
Context length: 16000
Threads: 32
GPU layers: 0
Max tokens: 30
Temperature: 0.2
Top-p sampling: 0.6

Available Methods:
  - create_model_path(): Download model files to a specified path.
  - create_llm(): Create and configure a language model for use.
  - __str__(): Return a string representation of the model's settings.
-----------------

openhermes-2.5-mistral-7b.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

orca-2-7b.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

llama-2-7b.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

In [ ]:
YAML_SysEngBench_string = '''
task: demo_sysengbench_zero_shot
dataset_path: rabell/SysEngBench
dataset_name: default  # Replace with specific subset name if applicable
description: "The following are multiple-choice questions (with answers) related to System Engineering."
test_split: test
output_type: multiple_choice
doc_to_text: |
  Question: {{question.strip()}}
  A. {{choiceA}}
  B. {{choiceB}}
  C. {{choiceC}}
  D. {{choiceD}}
  Answer:
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: answer
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
  - metric: acc_norm
    aggregation: mean
    higher_is_better: true
'''

# Write the YAML file
with open('/content/mmlu_SysEngBench.yaml', 'w') as f:
    f.write(YAML_SysEngBench_string)


In [ ]:
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 31.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.0-cp310-cp310-linux_x86_64.whl size=3486707 sha256=bd06f5988c1a0aebf01491c48b12976a09fc740556d9cbb2b1f5546525fc1b8a
  Stored in directory: /root/.cache/pip/wheels/81/72/ce/21e5bb4cc7c4a6799b225426e01bec5eb210e2457eb66321be
Successfully built llama-cpp-python


In [ ]:
# Install all necessary dependencies for llama_cpp.server
!pip install uvicorn fastapi sse-starlette starlette-context pydantic_settings

In [ ]:
# Start the server on localhost
!nohup python3 -m llama_cpp.server --model ./models/openhermes-2.5-mistral-7b.Q4_K_M.gguf --port 8000 --host 127.0.0.1 > server.log 2>&1 &


In [ ]:
# Check if any process is running on port 8090
!lsof -i :8000


COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 12650 root    8u  IPv4 422797      0t0  TCP *:8000 (LISTEN)


In [ ]:
# Run lm-eval using the OpenHermes GGUF model
!lm_eval \
    --model gguf \
    --model_args base_url=http://127.0.0.1:8000  \
    --include_path ./ \
    --tasks demo_sysengbench_zero_shot \
    --limit 3 \
    --output output/sys_eng_bench/ \
    --log_samples


2024-09-26 12:41:38.967293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 12:41:38.987134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 12:41:38.993127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 12:41:39.008215: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 12:41:40.576352: W tensorflow/comp